In [94]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [96]:
# import modules
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [124]:
# Load Mnist dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [125]:
# normalize the dataset
x_train, x_test = x_train / 255.0, x_test / 255.0

In [108]:
x_train[0].reshape(28*28)[600:650] # in the range 0 -> 1

array([0.99215686, 0.99215686, 0.99215686, 0.77647059, 0.31764706,
       0.00784314, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.07058824, 0.67058824, 0.85882353,
       0.99215686, 0.99215686, 0.99215686, 0.99215686, 0.76470588,
       0.31372549, 0.03529412, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.21568627, 0.6745098 ])

In [121]:
x_train.dtype, x_test.dtype

(dtype('float64'), dtype('float64'))

In [126]:
# Add channels dimension (grayscale -> 1 channel != rgb 3 channels)
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")
print(x_train.dtype, x_test.dtype)
print(x_train.shape, x_test.shape)

float32 float32
(60000, 28, 28, 1) (10000, 28, 28, 1)


In [127]:
# batch and shuffle the data
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

`tf.data.Dataset` in TensorFlow

In [140]:
dataset = tf.data.Dataset.from_tensor_slices([202.0,157.0,122.0])
dataset = dataset.map(lambda x : x / 255.0)
list(dataset.as_numpy_iterator())

[0.7921569, 0.6156863, 0.47843137]

In [144]:
dataset = tf.data.Dataset.range(100)
def dataset_fn(ds):
  return ds.filter(lambda x : x < 10) # even numbers
dataset = dataset.apply(dataset_fn)
list(dataset.as_numpy_iterator())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

`tf.data.Dataset.batch`

```
batch (
  batch_size,
  drop_remainder=False,
  num_parallel_calls=None,
  deterministic=None,
  name=None
)
```

In [152]:
dataset = tf.data.Dataset.range(10).batch(3, drop_remainder=True)
list(dataset.as_numpy_iterator())

[array([0, 1, 2]), array([3, 4, 5]), array([6, 7, 8])]

In [153]:
# Build tf.keras model using the Keras model subclassing API
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation="relu")
    self.flatten = Flatten()
    self.d1 = Dense(128, activation="relu")
    self.d2 = Dense(10) # output 10 classes

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of model
model = MyModel()

In [154]:
# Choose optimizer and loss function for training
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

# Choose metrics to measure the loss and accuracy of the model
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")
test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_loss")

In [155]:
# use tf.GradientTape to trauin the model
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_fn(labels, predictions)  # loss_function(true label, predicted labels) -> metrics loss
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels, predictions)

In [156]:
# test the model
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_fn(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [158]:
# Start trainin with epochs = 5
EPOCHS = 5
for epoch in range(EPOCHS):
  # reset the meitrcs at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
      f'Epoch {epoch + 1}, ' # epoch starts from 0
      f'Loss {train_loss.result()}, '
      f'Accuracy {train_accuracy.result() * 100}, ' # *100 -> percentage format
      f'Test Loss {test_loss.result()}, '
      f'Test Accuracy {test_accuracy.result() * 100}'
  )

Epoch 1, Loss 0.1337510198354721, Accuracy 95.94332885742188, Test Loss 0.05705123022198677, Test Accuracy 98.04000091552734
Epoch 2, Loss 0.04257063940167427, Accuracy 98.73333740234375, Test Loss 0.05799725651741028, Test Accuracy 98.0
Epoch 3, Loss 0.022624971345067024, Accuracy 99.2699966430664, Test Loss 0.06107187643647194, Test Accuracy 98.0999984741211
Epoch 4, Loss 0.012652875855565071, Accuracy 99.58999633789062, Test Loss 0.07004642486572266, Test Accuracy 98.04999542236328
Epoch 5, Loss 0.009029841981828213, Accuracy 99.69833374023438, Test Loss 0.08066807687282562, Test Accuracy 98.1199951171875


we do not save the result but reset, so we cannot visualize or plot any diagram